In [2]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import cycler
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
from functools import reduce
import matplotlib as mpl
#import geopandas as gpd
import requests
import pickle
from matplotlib.axis import Axis
from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [3]:
keys_dict = {'CENSUS': '24fc7d81b74510d599f702dbd408fb18e1466d81', 'BEA': '4B524457-8F97-44BF-9B03-1BD16890E169', 
             'BLS': '7cbe0223e97b4dea807629d438934e73'} #my API Keys
#wb is write bite to new file you're creating
with open('api_keys.pkl', 'wb') as outfile:
        pickle.dump(keys_dict, outfile)

In [4]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
api_key = keys_dict_2['CENSUS']

In [5]:
url_str= 'https://api.census.gov/data/2021/acs/acs5?key='+api_key

In [10]:
#davidson co
predicates= {} 
get_vars = ["NAME", 'GEO_ID', 'B01001_001E']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:037"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = ['NAME', 'GEO_ID', 'Population', 'Urban Area', 'thing']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
predicates = {}
get_vars = ["NAME", 'GEO_ID', 'B01001_001E']
predicates["get"] = ",". join(get_vars)
predicates["for"] = "urban area:*"
#predicates["in"] = "state:47"
data = requests.get(url_str, params = predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Urban Area']
uas = pd.DataFrame(columns = col_names, data = data.json()[1:], dtype=str)
df = pd.concat([df, uas], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [11]:
data = savename

In [12]:
data.head()

,NAME,GEO_ID,Population,Urban Area,thing
0,"Davidson County, Tennessee",0500000US47037,708490,47,037
0,"Abbeville, LA Urban Cluster (2010)",400C100US00037,18238,00037,NaN
1,"Abbeville, SC Urban Cluster (2010)",400C100US00064,4881,00064,NaN
2,"Abbotsford, WI Urban Cluster (2010)",400C100US00091,3997,00091,NaN
3,"Aberdeen, MS Urban Cluster (2010)",400C100US00118,3634,00118,NaN


In [ ]:
708490

In [13]:
nashville = data.loc[data['NAME'].str.contains('Nashville')]
nashville

,NAME,GEO_ID,Population,Urban Area,thing
2260,"Nashville, AR Urban Cluster (2010)",400C100US61192,3857,61192,NaN
2261,"Nashville, GA Urban Cluster (2010)",400C100US61219,4515,61219,NaN
2262,"Nashville, IL Urban Cluster (2010)",400C100US61246,3108,61246,NaN
2263,"Nashville-Davidson, TN Urbanized Area (2010)",400C100US61273,1108168,61273,NaN


In [14]:
#2020/2021
#the PL is a stand-in so that the change metrics will run
PL2021 = pd.read_csv('../../GNRC-Data-Integration/Outputs/2020PL.csv')#.set_index(['NAME', 'GEO_ID'])
ACS52021 = pd.read_csv('../../GNRC-Data-Integration/Outputs/2021ACS5YR.csv')#.set_index(['NAME', 'GEO_ID'])
Tiger2021 = pd.read_csv('../../GNRC-Data-Integration/Outputs/TIGERS2020.csv')#.set_index(['NAME', 'GEO_ID'])
Subject2021 = pd.read_csv('../../GNRC-Data-Integration/Outputs/2021Subject.csv')#.set_index(['NAME', 'GEO_ID'])
dfs2021 = [PL2021, Subject2021, ACS52021, Tiger2021]
twothousandtwentyone = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID'], how='outer'), dfs2021)

In [15]:
data = twothousandtwentyone.loc[twothousandtwentyone['NAME'] == 'Davidson County']
data.head()

,NAME,GEO_ID,Population,White Alone,White Alone %,Non-White Population: Race,Black or African American Alone,Black or African American Alone %,Black or African American Alone % Non-White Population,American Indian Alaska Native Alone,American Indian Alaska Native Alone %,American Indian Alaska Native Alone % Non-White Population,Asian Alone,Asian Alone %,Asian Alone % Non-White Population,Native Hawaiian Other Pacific Islander Alone,Native Hawaiian Other Pacific Islander Alone %,Native Hawaiian Other Pacific Islander Alone % Non-White Population,Some Other Race Alone,Some Other Race Alone %,Some Other Race Alone % Non-White Population,Two or More Races,Two or More Races %,Two or More Races % Non-White Population,Hispanic or Latino,Hispanic or Latino %,Not Hispanic or Latino,Not Hispanic or Latino %,Minority,Minority %,White Alone Not Hispanic or Latino,White Alone Not Hispanic or Latino %,Occupancy:Total Households,Occupancy:Occupied Units,Occupancy%:Occupied Units,Occupancy:Vacant Units,Occupancy%:Vacant Units,Group Quarters:Population,Group Quarters %:Population,Group Quarters:Institutionalized,Group Quarters %:Institutionalized,"Group Quarters:Institutionalized, Adults in Correctional Facility",Group Quarters % Institutionalized:Adults in Correctional Facility,Group Quarters %:Institutionalized Adults in Correctional Facility,"Group Quarters:Institutionalized, Juvenile Facility",Group Quarters % Institutionalized:Juvenile Facility,Group Quarters %:Institutionalized in Juvenile Facility,"Group Quarters:Institutionalized, Nursing or Skilled Nursing Facility",Group Quarters % Institutionalized:Nursing or Skilled Nursing Facility,Group Quarters %:Institutionalized in Nursing or Skilled Nursing Facility,"Group Quarters:Institutionalized, Other Facility",Group Quarters % Institutionalized:Other Facility,Group Quarters %:Institutionalized in Other Facility,Group Quarters:Noninstitutionalized,Group Quarters %:Noninstitutionalized,"Group Quarters:Noninstitutionalized, College or University Student Housing",Group Quarters % Noninstitutionalized:College or University Student Housing,Group Quarters %:Noninstitutionalized in College or University Student Housing,"Group Quarters:Noninstitutionalized, Military Quarters",Group Quarters % Noninstitutionalized:Military Quarters,Group Quarters %:Noninstitutionalized in Military Quarters,"Group Quarters:Noninstitutionalized, Other",Group Quarters % Noninstitutionalized:Other,Group Quarters %:Noninstitutionalized in Other,Mean Travel Time to Work,HealthCoverage:Total Series,HealthCoverage:None,HealthCoverage%: None,HealthCoverage: With Healthcare Coverage,HealthCoverage%: With Healthcare Coverage,HealthCoverage: With Public Healthcare Coverage,HealthCoverage%: With Public Healthcare Coverage,HealthCoverage: With Private Healthcare Coverage,HealthCoverage%: With Private Healthcare Coverage,Male Under 5,Female Under 5,Male 5 to 9,Female 5 to 9,Male 5 to 14,Female 5 to 14,Male 15 to 24,Female 15 to 24,Male 10 to 14,Female 10 to 14,Male 15 to 17,Female 15 to 17,Male 18 to 24,Female 18 to 24,Male 25 to 34,Female 25 to 34,Male 35 to 44,Female 35 to 44,Male 45 to 54,Female 45 to 54,Male 55 to 64,Female 55 to 64,Male 65 to 74,Female 65 to 74,Male 75 to 84,Female 75 to 84,Male 85 and Older,Female 85 and Older,Age:Under 5,Age %:Under 5,Age:5 to 9,Age%:5 to 9,Age:10 to 14,Age%:10 to 14,Age:15 to 17,Age%:15 to 17,Age:18 to 24,Age%:18 to 24,Age:25 to 34,Age%:25 to 34,Age:35 to 44,Age%:35 to 44,Age:45 to 54,Age%:45 to 54,Age:55 to 64,Age%:55 to 64,Age:65 to 74,Age%:65 to 74,Age:75 to 84,Age%:75 to 84,Age:85 and Older,Age%:85 and Older,Age:School Age 5 to 18,Age%:School Age 5 to 18,Age:Under 18,Age%:Under 18,Age:18 to 54,Age%:18 to 54,Age:18 to 64,Age%:18 to 64,Age:55 and Older,Age%:55 and Older,Age:65 and Older,Age%:65 and Older,Total Households,Family Households,Family Households: Married Couple Family,"Household Type%: Family, Married Couple Households",Family Households: Not Married Couple Family,"Househo

In [16]:
cols = ['Age:Under 18', 'Age%:Under 18', 'Disability:With Disability', 'Disability%:With Disability', 'Minority', 'Minority %',
        'Age:55 and Older', 'Age%:55 and Older', 'Age:65 and Older', 'Age%:65 and Older']

In [17]:
data = data[cols]
data.head()

,Age:Under 18,Age%:Under 18,Disability:With Disability,Disability%:With Disability,Minority,Minority %,Age:55 and Older,Age%:55 and Older,Age:65 and Older,Age%:65 and Older
10,147101.0,20.762608,78660.0,11.208401,329049.0,45.964011,168824.0,23.828706,87560.0,12.358678


In [18]:
data.to_csv('ECD2_15Request.csv', index = False)